# Finding the perfect coffee neighborhood in Toronto 

### Part 1 : Retrieve the postal code, borough and beighborhood data from the Wiki 

Import the required libraries 

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests as rq
import numpy as np 
import jsoｎ
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium
print('Libraries imported.')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Libraries imported.


Retreiving the URL and creating a Beautiful soup object

In [2]:
url ="http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = rq.get(url)
soup = bs(response.text)

Retrieve the required intormation into a list 

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

Create a dataframe with the list we created 

In [4]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.shape

(103, 3)

We get 103 rows of data in total 

### Part 2 : Retrieve the latitude and longitude for Toronto and each Borough

In [5]:
df_latlong = pd.read_csv(r'D:\Data.hhcheung\Desktop\Geospatial_Coordinates.csv') 

Joint the two dataframe

In [6]:
df_latlong.rename(columns ={'Postal Code':'PostalCode'}, inplace=True)
df_Merged = df.merge(df_latlong)
df_Merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [7]:
address = 'Toronto, Toronto'

geolocator = Nominatim(user_agent="CAN_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.65238435, -79.38356765.


Display the map showing all the neighborhood 

In [8]:
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Merged['Latitude'], df_Merged['Longitude'], df_Merged['Borough'], df_Merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Part 3 :　Explore the neighborhoods in Toronto

Adding the foursquares API

In [9]:
CLIENT_ID = 'E2AOTYTD2K2WYLRVYREBCR0PV31KVPGRNWHPALCXBXZ4O3QU' # your Foursquare ID
CLIENT_SECRET = 'IBZRGPABQN20UIVU1QYX5K3IF35TYODHOCFXFTEESX1TNYSM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E2AOTYTD2K2WYLRVYREBCR0PV31KVPGRNWHPALCXBXZ4O3QU
CLIENT_SECRET:IBZRGPABQN20UIVU1QYX5K3IF35TYODHOCFXFTEESX1TNYSM


Function : Return the nearby venue for a neighborhood

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    #Create a empty list 
    venues_list=[] 
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = rq.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Pass all the neighborhood into the function and create a new dataframe 

In [11]:
toronto_venues = getNearbyVenues(names=df_Merged['Neighborhood'],
                                   latitudes=df_Merged['Latitude'],
                                   longitudes=df_Merged['Longitude'])
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,DVP at York Mills,43.758899,-79.334099,Intersection
3,Parkwoods,43.753259,-79.329656,TTC Stop #09083,43.759655,-79.332223,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [12]:
toronto_venues.shape

(3658, 7)

Count the number of unique categories among the venue returned

In [13]:
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 324 uniques categories.


In [14]:
Venue_df = toronto_venues['Venue Category'].unique()
print(Venue_df)

['Park' 'Food & Drink Shop' 'Intersection' 'Bus Stop' 'Hockey Arena'
 'Portuguese Restaurant' 'Coffee Shop' 'Pizza Place' 'Playground'
 'Sporting Goods Shop' 'Bakery' 'Distribution Center' 'Historic Site'
 'Restaurant' 'Farmers Market' 'Chocolate Shop' 'Spa'
 'Mediterranean Restaurant' 'Breakfast Spot' 'Performing Arts Venue'
 'Gym / Fitness Center' 'Italian Restaurant' 'Pub' 'Liquor Store'
 'Dessert Shop' 'French Restaurant' 'Café' 'Tech Startup'
 'Thai Restaurant' 'Yoga Studio' 'Theater' 'Shoe Store' 'Animal Shelter'
 'Event Space' 'Pool' 'Gastropub' 'Art Gallery' 'Furniture / Home Store'
 'Brewery' 'Cosmetics Shop' 'Food Truck' 'Sandwich Place' 'Gym Pool' 'Bar'
 'Electronics Store' 'Sushi Restaurant' 'Bank' 'Beer Store' 'Skating Rink'
 'Pharmacy' 'Fast Food Restaurant' 'Hotel' 'Grocery Store'
 'Discount Store' 'Antique Shop' 'Tram Station' 'Athletics & Sports'
 'Dance Studio' 'Boutique' 'Vietnamese Restaurant' 'Greek Restaurant'
 'Fried Chicken Joint' 'Bowling Alley' 'Clothing Store

"Coffee Shop" & "Café" is the category we are looking for 

Show all the coffee shop retrieved 

In [155]:
Coffee_df = toronto_venues[toronto_venues['Venue Category'].isin(['Coffee Shop','Café'])]
Coffee_df

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
13,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
20,"Regent Park, Harbourfront",43.654260,-79.360636,Rooster Coffee,43.651900,-79.365609,Coffee Shop
22,"Regent Park, Harbourfront",43.654260,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop
23,"Regent Park, Harbourfront",43.654260,-79.360636,Starbucks,43.651613,-79.364917,Coffee Shop
24,"Regent Park, Harbourfront",43.654260,-79.360636,Sumach Espresso,43.658135,-79.359515,Coffee Shop
30,"Regent Park, Harbourfront",43.654260,-79.360636,Balzac's Coffee,43.649797,-79.359142,Coffee Shop
38,"Regent Park, Harbourfront",43.654260,-79.360636,ODIN Cafe + Bar,43.656739,-79.356503,Café
39,"Regent Park, Harbourfront",43.654260,-79.360636,Dark Horse Espresso Bar,43.653081,-79.357078,Coffee Shop
43,"Regent Park, Harbourfront",43.654260,-79.360636,Caffe Furbo,43.649970,-79.358849,Café


In [156]:
Coffee_df.shape

(439, 7)

Toronto consists of 439 coffee shops ＆ Café. 

Display the map that indicate the coffee shop 

In [157]:
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, venue, neighborhood in zip(Coffee_df['Venue Latitude'], Coffee_df['Venue Longitude'], Coffee_df['Venue'], Coffee_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [158]:
len(Coffee_df['Venue'].unique())

160

There are 160 unique coffee shop in total 

In [159]:
Coffee_Venue = Coffee_df.groupby('Venue').size().reset_index(name='counts')
Coffee_Venue.head()

,Venue,counts
0,808 Social,2
1,9bars,2
2,AGO Espresso Bar,1
3,Absolute Cafe,1
4,Aroma Espresso Bar,10


In [160]:
Coffee_Chain = Coffee_Venue[Coffee_Venue.counts >5]
Coffee_Chain

,Venue,counts
4,Aroma Espresso Bar,10
8,Balzac's Coffee,11
28,Café Plenty,7
44,Dineen @CommerceCourt,7
45,Dineen Coffee,7
76,Jimmy's Coffee,8
93,Maman,6
103,Mos Mos Coffee,7
104,NEO COFFEE BAR,7
111,Pilot Coffee Roasters,12


In [205]:
Coffee_Chain_List = Coffee_Chain['Venue']
Coffee_Chain_List

4         Aroma Espresso Bar
8            Balzac's Coffee
28               Café Plenty
44     Dineen @CommerceCourt
45             Dineen Coffee
76            Jimmy's Coffee
93                     Maman
103           Mos Mos Coffee
104           NEO COFFEE BAR
111    Pilot Coffee Roasters
125               Second Cup
130                Starbucks
150              Tim Hortons
153            Versus Coffee
Name: Venue, dtype: object

In [161]:
Coffee_Chain.shape

(14, 2)

There are 14 Coffee Chain with shop number larger than 5 

Finding the neigborhood with the most coffee shop 

In [162]:
Coffee_Neigborhood = Coffee_df.groupby('Neighborhood').size().reset_index(name='counts').sort_values(by=['counts'],ascending=False)
Coffee_Neigborhood.head()

,Neighborhood,counts
9,Central Bay Street,19
51,Ontario Provincial Government,17
59,St. James Town,17
28,"First Canadian Place, Underground city",16
13,"Commerce Court, Victoria Hotel",16


Central Bay Street are the neighborhood with the most coffee shop 

### Part 4 : Analysis the neighborhood 

It shows us how many venues there are in each Neighborhood:

In [164]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,14,14,14,14,14,14
"Alderwood, Long Branch",12,12,12,12,12,12
"Bathurst Manor, Wilson Heights, Downsview North",25,25,25,25,25,25
Bayview Village,11,11,11,11,11,11
"Bedford Park, Lawrence Manor East",39,39,39,39,39,39
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",7,7,7,7,7,7
"Brockton, Parkdale Village, Exhibition Place",83,83,83,83,83,83
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",26,26,26,26,26,26


#### Get dummies for the categories of these venues in each neighborhood.

In [165]:
indexNames =  toronto_venues[toronto_venues['Venue Category'].isin(['Coffee Shop','Café'])].index
toronto_venues_exclude_CoffeeShop = pd.DataFrame(data = toronto_venues.drop(indexNames, inplace=False))
toronto_venues_exclude_CoffeeShop.shape

(3219, 7)

In [166]:
toronto_venues.shape

(3658, 7)

In [167]:
Toronto_onehot=pd.get_dummies(toronto_venues_exclude_CoffeeShop[['Venue Category']], prefix='', prefix_sep='')
Toronto_onehot['Neighborhood']=toronto_venues_exclude_CoffeeShop['Neighborhood']

fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Souvlaki Shop,Spa

In [168]:
Toronto_onehot.shape

(3219, 322)

Analysis each neighborhood

In [169]:
toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Souvlaki Shop,Spa

Print the size of the new dataframe 

In [170]:
toronto_grouped.shape

(102, 322)

Create the new dataframe and display the top 10 venues for each neighborhood.

In [171]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [172]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Mediterranean Restaurant,Skating Rink,Seafood Restaurant,Badminton Court,Newsagent,Motorcycle Shop,Shopping Mall,Breakfast Spot,Sushi Restaurant,Supermarket
1,"Alderwood, Long Branch",Convenience Store,Pizza Place,Sandwich Place,Pub,Gym,Donut Shop,Gas Station,Skating Rink,Park,Women's Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Bank,Ice Cream Shop,Mobile Phone Shop,Shopping Mall,Bridal Shop,Supermarket,Sushi Restaurant,Diner,Sandwich Place
3,Bayview Village,Japanese Restaurant,Bank,Grocery Store,Intersection,Shopping Mall,Skating Rink,Chinese Restaurant,Playground,Dumpling Restaurant,Eastern European Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Restaurant,Sandwich Place,Indian Restaurant,Butcher,Intersection,Bagel Shop,Bakery,Bank,Sushi Restaurant


## Part 5: Cluster the new dataframe 

In [173]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 4, 4, 4, 6, 0, 6, 6, 6, 6])

In [174]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_Merged

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,5.0,Bus Stop,Intersection,Park,Food & Drink Shop,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,Intersection,Playground,Portuguese Restaurant,Pizza Place,Sporting Goods Shop,Hockey Arena,Park,Dog Run,Dessert Shop,Dim Sum Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6.0,Restaurant,Park,Bakery,Pub,Theater,Italian Restaurant,Performing Arts Venue,Breakfast Spot,Thai Restaurant,Gym Pool
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,6.0,Furniture / Home Store,Clothing Store,Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Dessert Shop,Sushi Restaurant,Athletics & Sports,Bowling Alley,Fried Chicken Joint
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,6.0,Sandwich Place,Park,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Yoga Studio,Burrito Place,Gastropub,Bookstore,Smoke Shop
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,4.0,Pharmacy,Playground,Home Service,Grocery Store,Shopping Mall,Bank,Skating Rink,Park,Women's Store,Dive Bar
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Home Service,Spa,Trail,Paper / Office Supplies Store,Bus Station,Comfort Food Restaurant,Comic Shop,Diner,Discount Store
7,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Japanese Restaurant,Gym,Caribbean Restaurant,Dessert Shop,Event Space,Farm,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,4.0,Pizza Place,Home Service,Gastropub,Flea Market,Athletics & Sports,Brewery,Intersection,Rock Climbing Spot,Fast Food Restaurant,Bank
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0.0,Hotel,Gastropub,Japanese Restaurant,Clothing Store,Sushi Restaurant,Spa,Theater,Furniture / Home Store,New American Restaurant,Falafel Restaurant


In [175]:
indexNames2 =  toronto_merged[toronto_merged['Cluster Labels'].isna()].index
toronto_merged_Clean = pd.DataFrame(data =  toronto_merged.drop(indexNames2, inplace=False))
toronto_merged_Clean.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,5.0,Bus Stop,Intersection,Park,Food & Drink Shop,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,Intersection,Playground,Portuguese Restaurant,Pizza Place,Sporting Goods Shop,Hockey Arena,Park,Dog Run,Dessert Shop,Dim Sum Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6.0,Restaurant,Park,Bakery,Pub,Theater,Italian Restaurant,Performing Arts Venue,Breakfast Spot,Thai Restaurant,Gym Pool
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,6.0,Furniture / Home Store,Clothing Store,Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Dessert Shop,Sushi Restaurant,Athletics & Sports,Bowling Alley,Fried Chicken Joint
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,6.0,Sandwich Place,Park,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Yoga Studio,Burrito Place,Gastropub,Bookstore,Smoke Shop


In [176]:
toronto_merged_Clean['Cluster Labels']= toronto_merged_Clean['Cluster Labels'].astype(int)
toronto_merged_Clean.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,5,Bus Stop,Intersection,Park,Food & Drink Shop,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar
1,M4A,North York,Victoria Village,43.725882,-79.315572,4,Intersection,Playground,Portuguese Restaurant,Pizza Place,Sporting Goods Shop,Hockey Arena,Park,Dog Run,Dessert Shop,Dim Sum Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6,Restaurant,Park,Bakery,Pub,Theater,Italian Restaurant,Performing Arts Venue,Breakfast Spot,Thai Restaurant,Gym Pool
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,6,Furniture / Home Store,Clothing Store,Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Dessert Shop,Sushi Restaurant,Athletics & Sports,Bowling Alley,Fried Chicken Joint
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,6,Sandwich Place,Park,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Yoga Studio,Burrito Place,Gastropub,Bookstore,Smoke Shop


In [177]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_Clean['Latitude'], toronto_merged_Clean['Longitude'], toronto_merged_Clean['Neighborhood'], toronto_merged_Clean['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1 

In [178]:
toronto_merged_Clean.loc[toronto_merged_Clean['Cluster Labels'] == 0, toronto_merged_Clean.columns[[1] + list(range(5, toronto_merged_Clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,North York,0,Japanese Restaurant,Gym,Caribbean Restaurant,Dessert Shop,Event Space,Farm,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
9,Downtown Toronto,0,Hotel,Gastropub,Japanese Restaurant,Clothing Store,Sushi Restaurant,Spa,Theater,Furniture / Home Store,New American Restaurant,Falafel Restaurant
15,Downtown Toronto,0,Cosmetics Shop,Seafood Restaurant,Hotel,Gastropub,American Restaurant,Restaurant,Italian Restaurant,Pizza Place,Furniture / Home Store,Beer Bar
20,Downtown Toronto,0,Hotel,Restaurant,Park,Beer Bar,Japanese Restaurant,Cocktail Bar,Bakery,Sandwich Place,Seafood Restaurant,Art Gallery
24,Downtown Toronto,0,Clothing Store,Art Gallery,Japanese Restaurant,Ramen Restaurant,Pizza Place,Burrito Place,Pub,Bar,French Restaurant,Middle Eastern Restaurant
30,Downtown Toronto,0,Hotel,Clothing Store,Theater,Beer Bar,Restaurant,Furniture / Home Store,Plaza,Arts & Crafts Store,Sushi Restaurant,Gym
33,North York,0,Clothing Store,Japanese Restaurant,Restaurant,Bank,Juice Bar,Fast Food Restaurant,Shopping Mall,Food Court,Bakery,Liquor Store
36,Downtown Toronto,0,Hotel,Boat or Ferry,Brewery,Japanese Restaurant,Park,Plaza,Scenic Lookout,Bar,Restaurant,Deli / Bodega
42,Downtown Toronto,0,Hotel,Gym,Restaurant,Seafood Restaurant,Japanese Restaurant,American Restaurant,Thai Restaurant,Theater,Lounge,Plaza
48,Downtown Toronto,0,Hotel,Beer Bar,Asian Restaurant,Concert Hall,Gastropub,Gym,Restaurant,Cosmetics Shop,Seafood Restaurant,Japanese Restaurant


#### Cluster 2

In [179]:
toronto_merged_Clean.loc[toronto_merged_Clean['Cluster Labels'] == 1, toronto_merged_Clean.columns[[1] + list(range(5, toronto_merged_Clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,1,Playground,Pool,Garden,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar


#### Cluster 3

In [180]:
toronto_merged_Clean.loc[toronto_merged_Clean['Cluster Labels'] == 2, toronto_merged_Clean.columns[[1] + list(range(5, toronto_merged_Clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,2,Pool,Martial Arts School,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant


#### Cluster 4

In [181]:
toronto_merged_Clean.loc[toronto_merged_Clean['Cluster Labels'] == 3, toronto_merged_Clean.columns[[1] + list(range(5, toronto_merged_Clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke Northwest,3,Hotel,Rental Car Location,Lounge,Dumpling Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant


#### Cluster 5

In [182]:
toronto_merged_Clean.loc[toronto_merged_Clean['Cluster Labels'] == 4, toronto_merged_Clean.columns[[1] + list(range(5, toronto_merged_Clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,4,Intersection,Playground,Portuguese Restaurant,Pizza Place,Sporting Goods Shop,Hockey Arena,Park,Dog Run,Dessert Shop,Dim Sum Restaurant
5,Etobicoke,4,Pharmacy,Playground,Home Service,Grocery Store,Shopping Mall,Bank,Skating Rink,Park,Women's Store,Dive Bar
6,Scarborough,4,Fast Food Restaurant,Home Service,Spa,Trail,Paper / Office Supplies Store,Bus Station,Comfort Food Restaurant,Comic Shop,Diner,Discount Store
8,East York,4,Pizza Place,Home Service,Gastropub,Flea Market,Athletics & Sports,Brewery,Intersection,Rock Climbing Spot,Fast Food Restaurant,Bank
10,North York,4,Pizza Place,Grocery Store,Gas Station,Restaurant,Metro Station,Mediterranean Restaurant,Playground,Fish Market,Latin American Restaurant,Sushi Restaurant
11,Etobicoke,4,Pizza Place,Theater,Mexican Restaurant,Gym,Hotel,Bank,Convenience Store,Restaurant,Women's Store,Doner Restaurant
17,Etobicoke,4,Park,Beer Store,Pizza Place,Pharmacy,Pet Store,Electronics Store,Gas Station,Convenience Store,Liquor Store,Donut Shop
18,Scarborough,4,Fast Food Restaurant,Pizza Place,Greek Restaurant,Sports Bar,Bank,Fried Chicken Joint,Restaurant,Beer Store,Community Center,Concert Hall
27,North York,4,Sandwich Place,Restaurant,Tennis Court,Pharmacy,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Bakery,Bank,Recreation Center
28,North York,4,Pizza Place,Bank,Ice Cream Shop,Mobile Phone Shop,Shopping Mall,Bridal Shop,Supermarket,Sushi Restaurant,Diner,Sandwich Place


#### Clsuter 6

In [183]:
toronto_merged_Clean.loc[toronto_merged_Clean['Cluster Labels'] == 5, toronto_merged_Clean.columns[[1] + list(range(5, toronto_merged_Clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,5,Bus Stop,Intersection,Park,Food & Drink Shop,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar
14,East York,5,Park,Athletics & Sports,Skating Rink,Bus Line,Bus Stop,Intersection,Curling Ice,Beer Store,Convenience Store,Eastern European Restaurant
22,Scarborough,5,Park,Convenience Store,Business Service,Donut Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
52,North York,5,Park,Pizza Place,Sushi Restaurant,Bank,Sandwich Place,Indian Restaurant,Trail,Sporting Goods Shop,Pet Store,Comic Shop
56,York,5,Gas Station,Convenience Store,Intersection,Sandwich Place,Discount Store,Park,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
57,North York,5,Gas Station,Baseball Field,Discount Store,Convenience Store,Falafel Restaurant,Donut Shop,Diner,Farmers Market,Distribution Center,Dive Bar
66,North York,5,Park,Bowling Alley,Convenience Store,Gym,Pet Store,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run
68,Central Toronto,5,Jewelry Store,Park,Gym / Fitness Center,Trail,Sushi Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
91,Downtown Toronto,5,Park,Trail,Playground,Candy Store,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar
101,Etobicoke,5,Playground,Gym / Fitness Center,Baseball Field,Park,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


#### Cluster 7 

In [184]:
toronto_merged_Clean.loc[toronto_merged_Clean['Cluster Labels'] == 6, toronto_merged_Clean.columns[[1] + list(range(5, toronto_merged_Clean.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,6,Restaurant,Park,Bakery,Pub,Theater,Italian Restaurant,Performing Arts Venue,Breakfast Spot,Thai Restaurant,Gym Pool
3,North York,6,Furniture / Home Store,Clothing Store,Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Dessert Shop,Sushi Restaurant,Athletics & Sports,Bowling Alley,Fried Chicken Joint
4,Queen's Park,6,Sandwich Place,Park,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Yoga Studio,Burrito Place,Gastropub,Bookstore,Smoke Shop
12,Scarborough,6,Breakfast Spot,Bar,Italian Restaurant,Burger Joint,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
13,North York,6,Gym,Restaurant,Intersection,Discount Store,Dim Sum Restaurant,Sushi Restaurant,Sandwich Place,Japanese Restaurant,Italian Restaurant,Supermarket
16,York,6,Field,Park,Sushi Restaurant,Korean Restaurant,Bakery,Sandwich Place,Italian Restaurant,Restaurant,Gastropub,Trail
19,East Toronto,6,Pub,Sandwich Place,Japanese Restaurant,Bar,Bakery,Breakfast Spot,Indie Movie Theater,Thai Restaurant,Asian Restaurant,Ice Cream Shop
21,York,6,Park,Women's Store,Hostel,Sporting Goods Shop,Beer Store,Bank,Bakery,Mexican Restaurant,Japanese Restaurant,Pizza Place
23,East York,6,Furniture / Home Store,Electronics Store,Sporting Goods Shop,Restaurant,Sports Bar,Brewery,Sandwich Place,Burger Joint,Department Store,Bank
25,Downtown Toronto,6,Grocery Store,Park,Indian Restaurant,Beer Store,Ice Cream Shop,Storage Facility,Latin American Restaurant,Karaoke Bar,Diner,Bakery


In [185]:
toronto_merged_Clean.loc[toronto_merged_Clean['Neighborhood'] =='Central Bay Street']

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Clothing Store,Art Gallery,Japanese Restaurant,Ramen Restaurant,Pizza Place,Burrito Place,Pub,Bar,French Restaurant,Middle Eastern Restaurant


Central Bay Street belongs to Cluster 1. 

#### Neighborhood that are in the same cluster group : 

In [197]:
Targeted_neighborhood =  toronto_merged_Clean.loc[toronto_merged_Clean['Cluster Labels'] == 0]['Neighborhood']
Targeted_neighborhood

7                                       Don Mills North
9                              Garden District, Ryerson
15                                       St. James Town
20                                          Berczy Park
24                                   Central Bay Street
30                             Richmond, Adelaide, King
33                         Fairview, Henry Farm, Oriole
36    Harbourfront East, Union Station, Toronto Islands
42             Toronto Dominion Centre, Design Exchange
48                       Commerce Court, Victoria Hotel
92                                       Enclave of M5E
97               First Canadian Place, Underground city
99                                 Church and Wellesley
Name: Neighborhood, dtype: object

In [204]:
Coffee_targeted_neighborhood = Coffee_Neigborhood[Coffee_Neigborhood['Neighborhood'].isin(Targeted_neighborhood)]
Coffee_targeted_neighborhood 

,Neighborhood,counts
9,Central Bay Street,19
59,St. James Town,17
28,"First Canadian Place, Underground city",16
13,"Commerce Court, Victoria Hotel",16
70,"Toronto Dominion Centre, Design Exchange",15
25,Enclave of M5E,15
55,"Richmond, Adelaide, King",14
29,"Garden District, Ryerson",13
32,"Harbourfront East, Union Station, Toronto Islands",12
4,Berczy Park,12


Don Mills North have the least number of coffee shop, it's recommended to start a coffee shop in that neighborhood 

#### Neighborhood with healthy competition (With least chain store) 

In [208]:
#Drop all chain store 
indexNames3 =  Coffee_df[Coffee_df['Venue'].isin(Coffee_Chain_List)].index
Coffee_df_without_chain = pd.DataFrame(data =   Coffee_df.drop(indexNames3, inplace=False))
#Group by Neighborhood
Coffee_Neigborhood_without_chain = Coffee_df_without_chain.groupby('Neighborhood').size().reset_index(name='counts').sort_values(by=['counts'],ascending=False)
Coffee_Neigborhood_without_chain.head()
#Filter out the cluster group 1 
Coffee_targeted_neighborhood_without_chain = Coffee_Neigborhood_without_chain[Coffee_Neigborhood_without_chain['Neighborhood'].isin(Targeted_neighborhood)]
Coffee_targeted_neighborhood_without_chain

,Neighborhood,counts
6,Central Bay Street,9
36,"Richmond, Adelaide, King",8
18,"First Canadian Place, Underground city",6
38,St. James Town,6
19,"Garden District, Ryerson",5
16,Enclave of M5E,4
20,"Harbourfront East, Union Station, Toronto Islands",4
9,"Commerce Court, Victoria Hotel",4
8,Church and Wellesley,4
46,"Toronto Dominion Centre, Design Exchange",4


Richmond, Adelaide, King have the healthiest competition environment.